# **Heuristic to select pipes to be replaced**

The heuristic in this code aims to find an approximate solution to the optimization problem:


>>$ min\:\: \sum_{i=0}^n c_ix_i$


subject to:

>>$\sum_{i=0}^n (1-x_i)br_i \leq target\:br$ \
\
$\sum_{i=0}^n (1-x_i)lr_i \leq target\:lr$ \
\
$\sum_{i=0}^n (1-x_i)si_i \leq target\:si$ \
\
$x_i \in \{0,1\}\: \: i=1..n$


where:

>>$c_i$ = cost of replacing pipe i\
$br_i$ = burst rate of pipe i \
$lr_i$ = leakage rate of pipe i\
$si_i$ = supply interruption time of pipe i\
$target\:br$ = target bust rate\
$target\:lr$ = target leakage rate\
$target\:si$ = target supply interruption time\
$x_i$ takes value 1 if pipe i is replaced and 0 otherwise




In [1]:
#load required packages
import pandas as pd
import numpy as np
import os
pd.set_option("display.max_rows", 50)
import warnings
warnings.filterwarnings("ignore")
import timeit
import datetime

## Input parameters

These will be the target values for the heuristics.

In [2]:
dataset = 'dataset2'

tests = \
  pd.read_excel(f'data/tests_{dataset}.xlsx')
try:
    test
except NameError:
    test = 0

#Start to measure execution time
start = datetime.datetime.now()

#Set right-hand side of the constraints
target_br = tests['target_burst_rate'].iloc[test]
target_lr = tests['target_leakage_rate'].iloc[test]
target_si = tests['target_supply_int'].iloc[test]



#Enter the number of properties
num_properties = 20000

#Enter the quantile
quantile = 0.9

#with or without adjustments
adjustment = 'wo_adjustment'




test += 1
print(test)

1


## Load the data and prepare it


In [3]:
#Create dataframe with the data

data = pd.read_excel(f'data/{dataset}.xlsx')

#Add column with expected leakage rate per day
data['Expected Leakage l/d'] = data['Expected Leakage l/h']*24

#Add column with average supply interruption in minutes
data['Avg supply int min'] = data['Expected Int to Supply Prop min']/num_properties

original_data = data.copy()


## Heuristics
This section contains the code of the heuristics, which is based on an algorithm that assigns points to each pipe depending on how efficient they are to close the feasibility gap.

**feasibility gap** is the difference between the actual value of the constraint and their target value. If the difference is less or equal than zero, which means that the constraint is not being violated, the feasibility gap is zero. 

For each constraint parameter, namely burst rate, leakage rate and average supply interruption time, the points are calculated as the pipe parameter value divided by the sum of the parameter value of all pipes. For instance, the burst rate points for pipe ***i*** would be:

>>$br\:points\:i = \frac{burst\: rate\: of\: pipe\: i}{sum\:burst\:rate\:all\:not\:selected\:pipes}$ 

In order to get the total points for each pipe properly, we must find the weights for each constraint parameter. The logic behind the weights relays on how "easy" filling the gap is. We measure the easiness as the ratio between the feasibility gap and the 90th quantile of constraint parameter. As we expect to fill the feasibility gap with the pipes that hold the highest values of constraints' parameters, the 90th quantile seems to be a sensible approximation of the easiness. Finally, the weights are adjusted to sum up one.  

Next, the weighted sum of the points is calculated and divided by the cost of replacing the pipe. This will give us a ranking with the more efficient pipes.

When selecting the last pipe, we don't want to select the pipe with the highest rank (based on the point rank depicted before), but the pipe able to fill the final gap at the lowest price. Therefore, an alternative ranking is made to select this pipe and make the heuristic more efficient.



In [4]:
#@title Heuristic Code
#Initial value of constraints
initial_br = sum(original_data['Expected Burst/year'])
initial_lr = sum(original_data['Expected Leakage l/d'])
initial_si = sum(original_data['Expected Int to Supply Prop min'])/num_properties

print('Intitial burst rate = {:,.1f}'.format(initial_br))
print('Intitial leakage rate = {:,.1f}'.format(initial_lr))
print('Intitial average supply int = {:,.1f}\n'.format(initial_si))

#Compute feasibility gap for each constraint
initial_br_gap = float(np.where(initial_br - target_br >0, initial_br - target_br, 0))
initial_lr_gap = float(np.where(initial_lr - target_lr >0, initial_lr - target_lr, 0))
initial_si_gap = float(np.where(initial_si - target_si >0, initial_si - target_si, 0))

print('Intitial burst rate gap = {:,.1f}'.format(initial_br_gap))
print('Intitial leakage rate gap = {:,.1f}'.format(initial_lr_gap))
print('Intitial supply interruption gap = {:,.1f}\n'.format(initial_si_gap))

#Get the number of constraints
#Some of the constraints are left without effect by fising the RHS way above the
  #intial value of the constraints' parameters. This is made for analysis
  #purpose
count_br = np.where(initial_br_gap >0, 1, 0)
count_lr = np.where(initial_lr_gap >0, 1, 0)
count_si = np.where(initial_si_gap >0, 1, 0)

n_real_constraints = count_br + count_lr + count_si


#Get the quantile for the constraints' parameters data
quantile_br = original_data['Expected Burst/year'].quantile(quantile)
quantile_lr = original_data['Expected Leakage l/d'].quantile(quantile)
quantile_si = original_data['Avg supply int min'].quantile(quantile)

#Get the ratio for each constraint as the division between the initial gap and 
#the desired quantile. Those constraints with a low quantile in relation to 
#their initial gap would have the highest ratio. It means that more attention 
#should be pointed towards them.

br_ratio = initial_br_gap/quantile_br
lr_ratio = initial_lr_gap/quantile_lr
si_ratio = initial_si_gap/quantile_si

#Computhe weights
ratios_sum = br_ratio + lr_ratio + si_ratio

br_w = br_ratio/ratios_sum
lr_w = lr_ratio/ratios_sum
si_w = si_ratio/ratios_sum

#Start with 0 pipes selected
not_selected_pipes = original_data.copy()

#Compute points for each pipe and constraint parameters. Pipes with higher  
  #values of the constraint parameters get higher points as they can close the 
  #gap easily

not_selected_pipes['burst_points'] = \
  100*not_selected_pipes['Expected Burst/year']/\
  sum(not_selected_pipes['Expected Burst/year'])

not_selected_pipes['leakage_points'] = \
  100*not_selected_pipes['Expected Leakage l/d']/\
  sum(not_selected_pipes['Expected Leakage l/d'])

not_selected_pipes['supply_points'] = \
  100*not_selected_pipes['Avg supply int min']/\
  sum(not_selected_pipes['Avg supply int min'])

#Get total points for each pipes by computing the weighted sum of each  
  #constraint points and dividing by the cost of replacing the pipes
not_selected_pipes['total_points'] = \
  (not_selected_pipes['burst_points']*br_w + \
   not_selected_pipes['leakage_points']*lr_w + \
   not_selected_pipes['supply_points']*si_w)/not_selected_pipes['Cost']

#Sort the pipes according to total points descending order
not_selected_pipes = not_selected_pipes.sort_values(by = ['total_points'], \
                                                    ascending= False)

#Get initial ranking as a reference to measure single ranking performance
initial_ranking = not_selected_pipes.copy()

#Get the constraints gap
br_gap = initial_br_gap
lr_gap = initial_lr_gap
si_gap = initial_si_gap

#Start the count 
i= 0

#Start with 0 pipes selected
selected_pipes = pd.DataFrame()

#Loop to select pipes
#Loop will continue until all the gaps have been filled
while br_gap > 0 or lr_gap > 0 or si_gap >0:
    

# #####################  FIRST ADJUSTMENT STEP STARTS  ###########################
#   #----------------------GET THE BEST LAST PIPE---------------------------------
#   #If the gaps can be filled with one pipe, it is preferable to select the 
#   #one with the minimum cost and not the one with the highest points, which
#   #may be different

#   first_pipe = not_selected_pipes['Expected Burst/year'].iloc[0]> br_gap and\
#     not_selected_pipes['Expected Leakage l/d'].iloc[0]> lr_gap and\
#     not_selected_pipes['Avg supply int min'].iloc[0]> si_gap

#   second_pipe = not_selected_pipes['Expected Burst/year'].iloc[1]> br_gap and\
#     not_selected_pipes['Expected Leakage l/d'].iloc[1]> lr_gap and\
#     not_selected_pipes['Avg supply int min'].iloc[1]> si_gap

    first_pipe = False
  
    if first_pipe:
        a =1
        
#     #Make a copy of the not_selected_pipes
#     alternative_ranking = not_selected_pipes.copy()
    
#     #Add a column to find pipes which can fullfill the gaps by their own
#     alternative_ranking['sufficient_pipe'] = ""
    
#     for j in range(alternative_ranking.shape[0]):
      
#       #If a pipe can fill the gaps, it assigns value Trues and False otherwise
#       alternative_ranking['sufficient_pipe'].iloc[j] = \
#         np.where(not_selected_pipes['Expected Burst/year'].iloc[j]> br_gap and \
#         not_selected_pipes['Expected Leakage l/d'].iloc[j]> lr_gap and \
#         not_selected_pipes['Avg supply int min'].iloc[j]> si_gap, True, False) 
    
#     #Dicth the pipes that cannot fill the gaps by their own
#     alternative_ranking = \
#       alternative_ranking[alternative_ranking['sufficient_pipe']==True]
    
    
#     #Get the pipes in ascending cost order
#     alternative_ranking = alternative_ranking.sort_values(by = ['Cost'])
    
#     #Attach the pipe with the lowest cost to selected pipes
#     selected_pipes = \
#       selected_pipes.append(not_selected_pipes[not_selected_pipes['Ref']==\
#                                                alternative_ranking.Ref.iloc[0]])
    
#     #Update the list of not selected pipes
#     not_selected_pipes = \
#       not_selected_pipes[not_selected_pipes.Ref != alternative_ranking.Ref.iloc[0]]
# #####################  FIRST ADJUSTMENT STEP FINISHES  #########################

#####################  HEURISTIC MAIN ALGORITHM STARTS  ######################## 
    else:
    
        #Select pipes according to the points criteria
        selected_pipes = selected_pipes.append(not_selected_pipes.iloc[0,:])

        #Update not selected pipes
        not_selected_pipes = not_selected_pipes.iloc[1:,:]

        #Update left-hand side of constraints 
        br = sum(not_selected_pipes['Expected Burst/year'])
        lr = sum(not_selected_pipes['Expected Leakage l/d'])
        si = sum(not_selected_pipes['Avg supply int min'])

        #Update constraints' gaps
        br_gap = float(np.where(br - target_br >0, br - target_br, 0))
        lr_gap = float(np.where(lr - target_lr >0, lr - target_lr, 0))
        si_gap = float(np.where(si - target_si >0, si - target_si, 0))

        #Updates quantile
        quantile_br = not_selected_pipes['Expected Burst/year'].quantile(quantile)
        quantile_lr = not_selected_pipes['Expected Leakage l/d'].quantile(quantile)
        quantile_si = not_selected_pipes['Avg supply int min'].quantile(quantile)

        #Updates ratios
        br_ratio = br_gap/quantile_br
        lr_ratio = lr_gap/quantile_lr
        si_ratio = si_gap/quantile_si

        ratios_sum = br_ratio + lr_ratio + si_ratio

        #Updates weights
        br_w = br_ratio/ratios_sum
        lr_w = lr_ratio/ratios_sum
        si_w = si_ratio/ratios_sum

        #Update points
        not_selected_pipes['burst_points'] = \
          100*not_selected_pipes['Expected Burst/year']/\
          sum(not_selected_pipes['Expected Burst/year']) 

        not_selected_pipes['leakage_points'] = \
          100*not_selected_pipes['Expected Leakage l/d']/\
          sum(not_selected_pipes['Expected Leakage l/d'])

        not_selected_pipes['supply_points'] = \
          100*not_selected_pipes['Avg supply int min']/\
          sum(not_selected_pipes['Avg supply int min'])

        not_selected_pipes['total_points'] = \
          (not_selected_pipes['burst_points']*br_w + \
           not_selected_pipes['leakage_points']*lr_w + \
           not_selected_pipes['supply_points']*si_w)/not_selected_pipes['Cost']

        #Update pipes ranking
        not_selected_pipes = \
          not_selected_pipes.sort_values(by = ['total_points'], ascending= False)
  
    #Update left-hand side of the constraints 
    br = sum(not_selected_pipes['Expected Burst/year'])
    lr = sum(not_selected_pipes['Expected Leakage l/d'])
    si = sum(not_selected_pipes['Avg supply int min'])

    #Update constraints' gap
    br_gap = float(np.where(br - target_br >0, br - target_br, 0))
    lr_gap = float(np.where(lr - target_lr >0, lr - target_lr, 0))
    si_gap = float(np.where(si - target_si >0, si - target_si, 0))

    #Update the number of iterations
    i+=1
####################  HEURISTIC MAIN ALGORITHM FINISHES  #######################

# ######################  SECOND ADJUSTMENT STEP STARTS  #########################
# #It could happen that when we select the last pipe, one or more previously 
#   #selected pipes turn to be in excess, increasing the total cost of the
#   #heuristic. This step detects those pipes and remove them.

# #Get the slack variables
# excess_br = target_br - br
# excess_lr = target_lr - lr
# excess_si = target_si - si

# #This loop find pipes in excess while no constraint is violated
# while excess_br >0 and excess_lr >0 and excess_si >0:
#   #Create an empty dataframe to store the pipes in excess
#   pipes_in_excess = pd.DataFrame()

#   #Loop through selected pipes to find pipes in excess
#   for pipe in range(selected_pipes.shape[0]):
  
#     excess_pipe = selected_pipes['Expected Burst/year'].iloc[pipe] <= excess_br and\
#       selected_pipes['Expected Leakage l/d'].iloc[pipe] <= excess_lr and\
#       selected_pipes['Avg supply int min'].iloc[pipe] <= excess_si
    
#     #If a pipe is identified to be in excess, it is added to the list
#     if excess_pipe:
#       pipes_in_excess = pipes_in_excess.append(selected_pipes.iloc[pipe,:])

#   #If no excess pipes are identified, the loop breaks
#   if pipes_in_excess.empty:
#     break

#   #If excess pipes are identified, we removed them from selected pipes and 
#    # placed them in not selected pipes
#   else:
#     #Remove the pipe in excess with the lowest cost from selected pipes
#     pipes_in_excess = \
#       pipes_in_excess.sort_values(by = ['Cost'], ascending= False).iloc[0,:]
    
#     selected_pipes = selected_pipes[selected_pipes.Ref != pipes_in_excess.Ref]

#     not_selected_pipes = not_selected_pipes.append(pipes_in_excess)
#     not_selected_pipes = \
#       not_selected_pipes.sort_values(by = ['total_points'], ascending= False)

#     #Update constraints' gap
#     br = sum(not_selected_pipes['Expected Burst/year'])
#     lr = sum(not_selected_pipes['Expected Leakage l/d'])
#     si = sum(not_selected_pipes['Avg supply int min'])

#     #Update slack variables
#     excess_br = target_br - br
#     excess_lr = target_lr - lr
#     excess_si = target_si - si
# ###################  SECOND ADJUSTMENT STEP FINISHES  ##########################

# ####################  THIRD ADJUSTMENT STEP STARTS  ############################
# # This step checks if the last pipe could be replaced by two pipes at a lower
#  # cost.

# #Identify pipes with half the cost of the last pipe. These pipes are candidates
#   # to replace the last pipe selected if another suitable pipe is found.
# alternative_ranking1 = \
#   not_selected_pipes[not_selected_pipes.Cost < selected_pipes.Cost.iloc[-1]/2]

# #Get the pipes with a cost lower than the last pipe selected
# alternative_ranking2 = \
#   not_selected_pipes[not_selected_pipes.Cost < selected_pipes.Cost.iloc[-1]]


# #Verify if pipes with a cost lower than half the price of the last selected
#   #pipe has been found and that more than one pipe with price lower than the
#   # last pipe have also been found. More than one is required as 
#   # alternative_ranking2 would naturally include the pipes in 
#   # alternative_ranking1
 
# if not alternative_ranking1.empty or alternative_ranking2.shape[0]>1:
  
#   #Get the actual cost of the solution
#   cost = sum(selected_pipes.Cost)
  
#   #Create an empty dataframe to store the candiates pipes to replace the last
#     #selected pipe.
#   merged_pipes = pd.DataFrame()
#   candidates_out = pd.DataFrame()
#   candidates_in = pd.DataFrame()
#   for k in range(alternative_ranking1.shape[0]):
#     #Store the candidate number one 
#     merged_pipes = merged_pipes.append(alternative_ranking1.iloc[k,:])
  
#     #Create another list without the candidate number one
#     alternative_ranking3 = \
#       alternative_ranking2[~alternative_ranking2['Ref'].isin(merged_pipes.Ref)]
    
#     #Filter the list with the pipes that in conjuction with the candidate 
#       #number one can fill the gap at a lower cost than the last selected pipe
#     alternative_ranking3 = \
#       alternative_ranking3[alternative_ranking3.Cost < \
#                           selected_pipes.Cost.iloc[-1] - sum(merged_pipes.Cost)]

#     required_br = \
#       -(target_br - br - selected_pipes['Expected Burst/year'].iloc[-1])
#     required_lr = \
#       -(target_lr - lr - selected_pipes['Expected Leakage l/d'].iloc[-1])
#     required_si = \
#       -(target_si - si - selected_pipes['Avg supply int min'].iloc[-1])

#     alternative_ranking3 = \
#       alternative_ranking3[(alternative_ranking3['Expected Burst/year'] > \
#         required_br - merged_pipes['Expected Burst/year'].iloc[0]) &\
#       (alternative_ranking3['Expected Leakage l/d'] > \
#         required_lr - merged_pipes['Expected Leakage l/d'].iloc[0]) &\
#       (alternative_ranking3['Avg supply int min'] > \
#         required_si - merged_pipes['Avg supply int min'].iloc[0])]
  
#     #If a suitable second candidate is found, the one with the lowest cost is
#      # choosen
#     if not alternative_ranking3.empty:
#       alternative_ranking3 = alternative_ranking3.sort_values(by=['Cost'])
      
#       merged_pipes = merged_pipes.append(alternative_ranking3.iloc[0,:])

#       replaced_pipe = selected_pipes.iloc[-1,:].to_frame().T

#       cost1 = sum(selected_pipes.Cost) + sum(merged_pipes.Cost) -\
#               sum(replaced_pipe.Cost)
      
#       #If the cost improves, the we'd found candidates
#       if cost1 < cost:
#         cost = cost1
#         candidates_in = merged_pipes
#         candidates_out = replaced_pipe

#   #Update selected and not selected pipes
#   if candidates_in.shape[0]==2:
#     selected_pipes = selected_pipes.iloc[:-1,:]
#     selected_pipes = selected_pipes.append(candidates_in)

#     not_selected_pipes = candidates_out.append(not_selected_pipes)
#     not_selected_pipes = \
#       not_selected_pipes[~not_selected_pipes['Ref'].isin(candidates_in.Ref)]

#     #Update constraints' gap
#     br = sum(not_selected_pipes['Expected Burst/year'])
#     lr = sum(not_selected_pipes['Expected Leakage l/d'])
#     si = sum(not_selected_pipes['Avg supply int min'])

#     #Update slack variables
#     excess_br = target_br - br
#     excess_lr = target_lr - lr
#     excess_si = target_si - si
# #####################  THIRD ADJUSTMENT STEP FINISHES  #########################   

# ######################  FOURTH ADJUSTMENT STEP STARTS  #########################
# #This step checks if one of the selected pipes, together with the last selected
#  #pipe, could be replaced by one of the not selected pipes at a lower cost

# #Return last pipe to an alterntive not_selected dataframe
# selected_pipes1 = selected_pipes.iloc[:-1,:]
# not_selected_pipes1 = not_selected_pipes.append(selected_pipes.iloc[-1,:])


# #Update left-hand side of the constraints 
# br1 = sum(not_selected_pipes1['Expected Burst/year'])
# lr1 = sum(not_selected_pipes1['Expected Leakage l/d'])
# si1 = sum(not_selected_pipes1['Avg supply int min'])

# #Update constraints' gap
# br_gap1 = float(np.where(br1 - target_br >0, br1 - target_br, 0))
# lr_gap1 = float(np.where(lr1 - target_lr >0, lr1 - target_lr, 0))
# si_gap1 = float(np.where(si1 - target_si >0, si1 - target_si, 0))

# #From not_selected_pipes filter the ones which could fullfill the gap filled by
#   #the last selected pipe 
# not_selected_pipes1 = \
#   not_selected_pipes[(not_selected_pipes['Expected Burst/year'] >= br_gap1)\
#                     & (not_selected_pipes['Expected Leakage l/d']>= lr_gap1)\
#                     & (not_selected_pipes['Avg supply int min']>= si_gap1)]

# if not not_selected_pipes1.empty:

#   #If one or more pipe exist, sort them by cost
#   not_selected_pipes1 = not_selected_pipes1.sort_values(by=['Cost'])
  
#   #Get the actual cost
#   cost = sum(selected_pipes.Cost)
  
#   #Empty dataframe to store candidates to be pulled out of selected pipe
#   candidates_out = pd.DataFrame()
#   candidates_in = pd.DataFrame
  
#   #Loop to appraise the pipes to be pushed in the selected pipes
#   for h in range(not_selected_pipes1.shape[0]):
    
#     #Get the surplus the pipe would generate if pushed into the selected lits
#     br_excess1 = \
#       br1 - target_br - not_selected_pipes1['Expected Burst/year'].iloc[h]
    
#     lr_excess1 = \
#       lr1 - target_lr - not_selected_pipes1['Expected Leakage l/d'].iloc[h]
    
#     si_excess1 = \
#       si1 - target_si - not_selected_pipes1['Avg supply int min'].iloc[h]

#     #Check if with the surplus generated, some selected pipes are in excess 
#     selected_pipes2 = \
#       selected_pipes1[(selected_pipes1['Expected Burst/year']<= -br_excess1)\
#                     & (selected_pipes1['Expected Leakage l/d']<= -lr_excess1)\
#                     & (selected_pipes1['Avg supply int min']<= -si_excess1)\
#                     & (selected_pipes1['Cost']>\
#                         not_selected_pipes1['Cost'].iloc[h]-\
#                         selected_pipes['Cost'].iloc[-1])]

#     #If some selected pipes happen to be in excess, sort them by cost
#     if not selected_pipes2.empty:
      
#       selected_pipes2 = \
#         selected_pipes2.sort_values(by=['Cost'], ascending= False)
       
#       #Get the total cost after adding and removing the intended pipes
#       cost1 = sum(selected_pipes.Cost) - selected_pipes2.Cost.iloc[0] -\
#               selected_pipes.Cost.iloc[-1] + not_selected_pipes1['Cost'].iloc[h]
      
#       #If cost improves, pipes are store in candidates pipe are added to the 
#         #respective list

#       if cost > cost1:
#         cost = cost1
#         candidates_in = not_selected_pipes1.iloc[h,:]
        
#         if candidates_out.shape[0]>1:
#           candidates_out = selected_pipes2.iloc[0,:]
#         else:
#           candidates_out = candidates_out.append(selected_pipes2.iloc[0,:])
        
# #Add the last selected pipe to the candidates out list
# candidates_out = candidates_out.append(selected_pipes.iloc[-1,:])

# #If two candidates have been found, update selected and not selected pipes
# if candidates_out.shape[0]==2:
#   selected_pipes = selected_pipes[~selected_pipes.Ref.isin(candidates_out.Ref)]
#   selected_pipes = selected_pipes.append(candidates_in)

#   not_selected_pipes = \
#     not_selected_pipes[~not_selected_pipes.Ref.isin([candidates_in.Ref])]
  
#   not_selected_pipes = candidates_out.append(not_selected_pipes)
# #####################  FOURTH ADJUSTMENT STEP FINISHES  ########################

print(f'Number of pipes selected = {selected_pipes.shape[0]}\n')

print('Total cost = £ {:,.0f}'.format(sum(selected_pipes['Cost'])))

Intitial burst rate = 6,121.8
Intitial leakage rate = 4,436,418.6
Intitial average supply int = 22,612.7

Intitial burst rate gap = 221.8
Intitial leakage rate gap = 0.0
Intitial supply interruption gap = 22,312.7

Number of pipes selected = 22269

Total cost = £ 2,181,776,278


## Selected pipes summary

In [5]:
selected_pipes.sum()

Avg supply int min                 2.231276e+04
Cost                               2.181776e+09
Expected Burst/year                5.443239e+03
Expected Int to Supply Prop min    4.462551e+08
Expected Leakage l/d               3.943190e+06
Expected Leakage l/h               1.642996e+05
Ref                                1.111651e+09
burst_points                       2.198325e+02
leakage_points                     2.195213e+02
supply_points                      4.281899e+02
total_points                       3.570526e-03
dtype: float64

In [6]:
selected_pipes
file_name = f'results_{dataset}/{adjustment}/selected_pipes_{adjustment}_br{target_br}_lr{target_lr}_si{target_si}.csv'
selected_pipes.to_csv(file_name, index = False)

## Not selected pipes summary

In [7]:
not_selected_pipes.sum()

Ref                                3.888399e+09
Cost                               3.550618e+09
Expected Burst/year                6.785295e+02
Expected Leakage l/h               2.055118e+04
Expected Int to Supply Prop min    5.999748e+06
Expected Leakage l/d               4.932283e+05
Avg supply int min                 2.999874e+02
burst_points                       1.000000e+02
leakage_points                     1.000000e+02
supply_points                      1.000000e+02
total_points                       0.000000e+00
dtype: float64

In [8]:
not_selected_pipes
file_name = f'results_{dataset}/{adjustment}/not_selected_pipes_{adjustment}_br{target_br}_lr{target_lr}_si{target_si}.csv'
not_selected_pipes.to_csv(file_name, index = False)

In [9]:
#End execution time measurement
end = datetime.datetime.now()
execution_time = end - start
execution_time = execution_time.total_seconds()

print(execution_time)

2309.526119


## Comparison with Optimal Solution

In [11]:
#@title Code for comparison
#Heuristics cost
heuristics_cost = sum(selected_pipes['Cost'])
num_pipes_heuristic = selected_pipes.shape[0]

#Read optimal solution
optimal_solution = \
  pd.read_csv(f'data/{dataset}_optimal_solutions/Results_br{target_br}_lr{target_lr}_si{target_si}.csv')

#Get optimal solution from Xpress
optimal_solution_cost = sum(optimal_solution['cost_GBP'])
num_optimal_pipes = optimal_solution.shape[0]
optimal_pipes = list(optimal_solution.pipe_number)
xpress_time = np.mean(optimal_solution.execution_time_sec)

#Get the number of pipes selected correctly
bolean_filter = selected_pipes.Ref.isin(optimal_pipes)
correct_selected_pipes = selected_pipes[bolean_filter]
num_correct_selected_pipes = correct_selected_pipes.shape[0]

#Compute the accuracy in terms of pipes selected correctly
pipes_accuracy = num_correct_selected_pipes/num_optimal_pipes

#Get the cost difference
cost_difference = heuristics_cost - optimal_solution_cost
heuristics_cost_diff = (cost_difference/optimal_solution_cost)*100

#Print results
print('Heuristic Cost (£) = {:,.0f}'.format(heuristics_cost))
print('Optimal Cost (£) = {:,.0f}'.format(optimal_solution_cost))
print('Heuristic Cost Difference (%) = {:.2f}'.format(heuristics_cost_diff))
print('Number of Optimal Pipes  = {:,.0f}'.format(num_optimal_pipes))
print('Accuracy in terms of pipes = {:.2f}'.format(pipes_accuracy))


Heuristic Cost (£) = 2,181,776,278
Optimal Cost (£) = 198,335
Heuristic Cost Difference (%) = 1099944.35
Number of Optimal Pipes  = 2
Accuracy in terms of pipes = 0.50


In [ ]:
#@title Pass results to cvs
id = f'br{target_br}_lr{target_lr}_si{target_si}'
file_name = f'results_{dataset}/heuristic_accuracy_{dataset}_{adjustment}.csv'

if os.path.isfile(file_name) == True:
    a=1
else:
    file = open(file_name,'x')
    file.writelines('id,target_burst_rate,target_leakage_rate,target_supply_int,\
    optimal_cost_gbp,heuristic_cost_gbp,cost_difference_gbp,heuristic_cost_excess_%,\
    n_optimal_pipes,n_pipes_heuristic,correct_pipes_selected,pipe_accuracy,Xpress_time_sec,\
    heuristics_time_sec,n_constraints\n')
    file.close()

file = open(file_name,'a')
file.writelines(f'{id},{target_br},{target_lr},{target_si},{optimal_solution_cost},\
{heuristics_cost},{cost_difference},{heuristics_cost_diff},{num_optimal_pipes},{num_pipes_heuristic},\
{num_correct_selected_pipes},{pipes_accuracy},{xpress_time},{execution_time},\
{n_real_constraints}\n')
file.close()

In [ ]:
print(target_br), print(target_lr), print(target_si) 
print(optimal_solution_cost), print(heuristics_cost), print(cost_difference), print(heuristics_cost_diff)
print(num_optimal_pipes), print(num_correct_selected_pipes)
print(pipes_accuracy), print(xpress_time), print(execution_time)
print(n_real_constraints)